In [8]:
! pip install bertopic

/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [9]:
import json

with open("/kaggle/input/slic-data-cases/indices.json", 'r') as json_file:
    data_represent = json.load(json_file)

case_a = []
case_b = []

misclassified = []

tag = 1

case_a_file = "/GPT 4 Vision/HARM_C/CASE 3"
case_b_file = "/GPT 4 Vision/HARM_C/CASE 4"

for itemize in data_represent[case_a_file]:
    case_a.append(itemize)
    misclassified.append(itemize)

for itemize in data_represent[case_b_file]:
    case_b.append(itemize)
    misclassified.append(itemize)

assert len(misclassified) == len(case_a) + len(case_b)
print(len(misclassified), len(case_a), len(case_b))

case_a_file = "/GPT 4 Vision/HARM_P/CASE 3"
case_b_file = "/GPT 4 Vision/HARM_P/CASE 4"

for itemize in data_represent[case_a_file]:
    case_a.append(itemize)
    misclassified.append(itemize)

for itemize in data_represent[case_b_file]:
    case_b.append(itemize)
    misclassified.append(itemize)

assert len(misclassified) == len(case_a) + len(case_b)
print(len(misclassified), len(case_a), len(case_b))

33 13 20
92 25 67


In [10]:
import json

docs = []

filename = '/kaggle/input/gpt-4o-outputs/gpt-4o explanations/gpt_4o_harm_c_def_ocr_in_ex_out.txt'
with open(filename, 'r') as file: 
    content = file.read()

counter = 0
for sample in content.split("##########"):
    splitted_sample = sample.split("----------")
    if len(splitted_sample) == 2:
        counter += 1
        json_image = json.loads(splitted_sample[0])["image"].split("/")[-1]
        output_image = splitted_sample[1]
        if json_image in misclassified:
            docs.append(output_image)
            assert int(json.loads(splitted_sample[0])["label"]) == tag

print(counter)

filename = '/kaggle/input/gpt-4o-outputs/gpt-4o explanations/gpt_4o_harm_p_def_ocr_in_ex_out.txt'
with open(filename, 'r') as file: 
    content = file.read()

counter = 0
for sample in content.split("##########"):
    splitted_sample = sample.split("----------")
    if len(splitted_sample) == 2:
        counter += 1
        json_image = json.loads(splitted_sample[0])["image"].split("/")[-1]
        output_image = splitted_sample[1]
        if json_image in misclassified:
            docs.append(output_image)
            assert int(json.loads(splitted_sample[0])["label"]) == tag

print(counter)

assert len(docs) == len(misclassified)
print(len(docs))

354
355
92


In [11]:
from bertopic import BERTopic

from bertopic.representation import KeyBERTInspired, VisualRepresentation
from bertopic.backend import MultiModalBackend
from sklearn.cluster import KMeans

from sklearn.feature_extraction.text import CountVectorizer

from umap import UMAP

cluster_model = KMeans(n_clusters=2)

umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=42)

# Vectorizer
vectorizer_model = CountVectorizer(stop_words="english")

# Train our model with images only
topic_model = BERTopic(
    umap_model=umap_model,
    vectorizer_model=vectorizer_model,
    hdbscan_model=cluster_model
)
topics, probs = topic_model.fit_transform(documents=docs)

In [12]:
from IPython.display import HTML


# Extract dataframe
df = topic_model.get_topic_info()

# Visualize the images
HTML(df.to_html(escape=False))


,Topic,Count,Name,Representation,Representative_Docs
0,0,53,0_notharmful_meme_harm_individual,"[notharmful, meme, harm, individual, does, cause, defamation, organization, humorous, humorously]","[\nnot-harmful - This meme is a humorous exchange and does not appear to cause any harm, such as defamation, mental abuse, or emotional\n, \nnot-harmful - The meme does not target or defame a specific individual, organization, or community, nor does it cause mental abuse or emotional\n, \nnot-harmful - The meme does not contain any content that would cause harm to an individual, organization, community, or society. It appears to\n]"
1,1,39,1_political_notharmful_meme_harm,"[political, notharmful, meme, harm, causing, individuals, organizations, individual, society, organization]","[\nnot-harmful - The meme expresses a humorous political opinion about taxes without causing harm or defamation to any individual, organization, community, or society\n, \nnot-harmful - The meme expresses a political opinion without causing direct harm, defamation, or emotional disturbance to any individual, organization, or society\n, \nnot-harmful - The meme expresses a political viewpoint without targeting or causing harm to individuals, organizations, communities, or society. It encourages political involvement\n]"


In [13]:
grouped_topics = {topic: [] for topic in set(topics)}
for index, topic in enumerate(topics):
    grouped_topics[topic].append(index)
print(grouped_topics)

{0: [0, 1, 3, 4, 5, 6, 7, 8, 10, 11, 12, 14, 15, 16, 17, 18, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 35, 41, 43, 45, 46, 48, 50, 51, 52, 53, 58, 61, 66, 67, 74, 76, 77, 79, 80, 81, 83, 87, 88, 91], 1: [2, 9, 13, 19, 33, 34, 36, 37, 38, 39, 40, 42, 44, 47, 49, 54, 55, 56, 57, 59, 60, 62, 63, 64, 65, 68, 69, 70, 71, 72, 73, 75, 78, 82, 84, 85, 86, 89, 90]}


In [14]:
print(case_a_file, case_b_file, sep="\n")

a = 0
b = 0

assert len(grouped_topics) == 2
assert 0 in grouped_topics and 1 in grouped_topics

for item in grouped_topics[0]:
    if misclassified[item] in case_a:
        a += 1
    else:
        assert misclassified[item] in case_b
        b += 1

print(int(round(a/(a+b), 2)*100), int(round(b/(a+b), 2)*100))

a = 0
b = 0

for item in grouped_topics[1]:
    if misclassified[item] in case_a:
        a += 1
    else:
        assert misclassified[item] in case_b
        b += 1

print(int(round(a/(a+b), 2)*100), int(round(b/(a+b), 2)*100))

/GPT 4 Vision/HARM_P/CASE 3
/GPT 4 Vision/HARM_P/CASE 4
26 74
28 72
